<p id="top" style="background-color:#1A2D45;font-family:newtimeroman; color:white;text-align:center;border-radius:10px;font-size:24px">We are all astronauts on spaceship Kaggle!</p>
<center>
    <img src="https://i.postimg.cc/Dy1zQGsq/image.png" style="border-radius:40px;font-size:24px;" alt="Abdirayimov Sardor">
</center>




In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")


for dirname, _, filenames in os.walk('/kaggle/input'):   
     for filename in filenames:
        name = os.path.join(dirname, filename)
        print(name)
        if "train.csv" in name:                   
            train = pd.read_csv(name)                
        elif "test.csv" in name:
            test = pd.read_csv(name)              
        elif "submission.csv" in name:
            sample = pd.read_csv(name)

<p id="descriptive" style="background-color:#546981 ;font-family:newtimeroman;color:white;font-size:24px;text-align:center;border-radius:10px;">First-to-Do</p>

In [ ]:
train.info()

<p id="descriptive" style="background-color:#546981 ;font-family:newtimeroman;color:white;font-size:24px;text-align:center;border-radius:10px;">Originally, train and test sets were one single dataset. In order to fill mssing values efficiently, joing both sets into one is very crucial!</p>

In [ ]:
great = pd.concat([train, test], ignore_index=True)

<p id="descriptive" style="background-color:#546981 ;font-family:newtimeroman;color:white;font-size:24px;text-align:center;border-radius:10px;">Key acpects from my perspectives:
<ul>    
<li>From Cabin column, we are creating three seperate columns by naming them as Deck, Cabin_num and Side respectively</li>
    
<li>From PassengerId column, which has zero missing values, we are creating two features such as Group and Group_num</li>
    
<li>From Name column, I seperate only Family name from First name and create new coumn called Family</li>
</ul></p>

In [ ]:
def upgrade_dataset(data):
    df = data.copy()
    df[['Deck','Cabin_num','Side']] = df['Cabin'].str.split("/", expand=True)
    df[["Group", "Group_num"]] = df['PassengerId'].str.split("_", expand=True)
    df[['Name', 'Family']] = df['Name'].str.split(" ",expand=True)
    df.drop(["Cabin", "Name"], axis=1, inplace=True) # not sure to drop Name, it helps to find parent and child
    df[["Group", "Group_num"]] = df[["Group", "Group_num"]].astype(int)
    df.Cabin_num.loc[df.Cabin_num.notnull()] = df.Cabin_num.loc[df.Cabin_num.notnull()].astype(int)
    return df

In [ ]:
df = upgrade_dataset(great)

In [ ]:
import missingno 

missingno.matrix(df,figsize=(16,8), fontsize=12, color=(0.10, 0.18, 0.27)) #color in misingno is in RGB Percent
plt.show()

<p id="fact" style="background-color:#1A2D45;font-family:newtimeroman; color:white;text-align:center;border-radius:10px;font-size:24px">Fact about passengers under 13 y.o.</p>

<form action="/action_page.php">
  <fieldset>
    <legend>Fact:</legend>
    
   <div class="card">
    <p class="card-text">Passengers under 13 years old can not use services in the passenger liner</p>
     
  </div>
  </fieldset>
</form>




<p style="background-color:#546981;font-family:newtimeroman; color:white;text-align:center;border-radius:10px;font-size:24px">By using previous fact, I will replace missing values in all services of passengers under 13 y.o.  into 0.0</p>


In [ ]:
SERVICES = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
values = dict(zip(SERVICES, [0.0]*5))
values['VIP'] = False
df.loc[df.Age < 13] = df.loc[df.Age < 13].fillna(value=values)

<p id="Family" style="background-color:#1A2D45;font-family:newtimeroman; color:white;text-align:center;border-radius:10px;font-size:24px">Missings in the Family</p>

If there are missing values in Family feature, than folloling function searchs other rows with the same Group feature(PassengerId). Next, following function return most popular Family name from searched rows.
Otherwise, following function return random family names for solo passengers 

P.S.: Basicly similar technique will be used for other columns with missing values too!

In [ ]:
!pip install unique-names-generator

In [ ]:
from unique_names_generator import get_random_name

def put_name(item):
    fam_df = df.loc[df.Group == item]
    if len(fam_df) > 1:
        return fam_df.Family.describe().top
    else:
        random  = get_random_name().split()[1]
        return random


In [ ]:
df.Family.loc[df.Family.isnull()] = df.loc[df.Family.isnull()]['Group'].apply(put_name)

<p id="3features" style="background-color:#546981;font-family:newtimeroman; color:white;text-align:center;border-radius:10px;font-size:24px">CryoSleep & VIP & Services</p>

If pasenger use CrySleep, he or she can not use Services and most probably VIP feature too.

I am going to fill missing values in services based on cryosleep:
- CryoSleep == True => Services = 0.0;
- CrySleep == False => Services = mean or median values of feature

The same technique to missing values in Cryosleep;
- Services and/or VIP == 0.0 => Cryoslepp = True
- Services and/or VIP != 0.0 => CrySleep = False


In [ ]:
values = dict(zip(SERVICES, [0.0]*5))
df.loc[df.CryoSleep == True] = df.loc[df.CryoSleep == True].fillna(values)

<form action="/action_page.php">
  <fieldset>
    <legend>Fact:</legend>
    
   <div class="card">
    <p class="card-text">Passenger only from Europe (HomePlanet) Have CryoSleep and VIP equal True </p>
     
  </div>
    
  </fieldset>
</form>

<p id="HomePlanet" style="background-color:#1A2D45;font-family:newtimeroman; color:white;text-align:center;border-radius:10px;font-size:24px">Missings in the HomePlanet</p>

If there are missing values in HomePlanet feature, than folloling function searchs other rows with the same Group feature(PassengerId). Next, following function return most popular HomePlanet name from searched rows.

Otherwise, following function return the most popular HomePlanet name from main dataset 

In [ ]:
def get_home(item):
    data = df.loc[df.Family == item]
    if len(data.loc[data.HomePlanet.notnull()]) > 1:
        return data.HomePlanet.describe().top
    else:
        return df.HomePlanet.describe().top
    

In [ ]:
df.HomePlanet.loc[df.HomePlanet.isnull()] = df.loc[df.HomePlanet.isnull()]['Family'].apply(get_home)

<p id="Destination" style="background-color:#1A2D45;font-family:newtimeroman; color:white;text-align:center;border-radius:10px;font-size:24px">Missings in the Destination</p>

If there are missing values in Destination feature, than folloling function searchs other rows with the same Group feature(PassengerId). Next, following function return most popular Destination name from searched rows.

Otherwise, following function return the most popular Destination name from main dataset 


In [ ]:
def get_destination(item):
    data = df.loc[df.Family == item]
    if len(data) > 1:
        return data.Destination.describe().top
    else:
        return df.Destination.describe().top

In [ ]:
 df.Destination.loc[df.Destination.isnull()] = df.loc[df.Destination.isnull()]['Family'].apply(get_destination)

<p id="CryoSleep" style="background-color:#1A2D45;font-family:newtimeroman; color:white;text-align:center;border-radius:10px;font-size:24px">Missings in the CryoSleep</p>

<form action="/action_page.php">
  <fieldset>
    <legend>Fact:</legend>
    
   <div class="card">
    <p class="card-text">Mostly, Passengers who have CryoSleep did not use services
     
  </div>
    
  </fieldset>
</form>

So, If passenger has missing value in the CryoSleep column and if passenger does not use any services in the spaceship, than most probably that person\`s CrySleep is True. Otherwise, it is False

In [ ]:
def get_cryosleep(item):
    if item == 0.0:
        return True
    else:
        return False

In [ ]:
df.CryoSleep.loc[df.CryoSleep.isnull()] = df.loc[df.CryoSleep.isnull()][SERVICES].sum(axis=1).apply(get_cryosleep)

<p id="VIP" style="background-color:#1A2D45;font-family:newtimeroman; color:white;text-align:center;border-radius:10px;font-size:24px">Missings in the VIP</p>

If passenger spends money more than 2149.83(mean value) than that person uses VIP features.

Otherwise, pessenger does not use VIP.

In [ ]:
df.loc[(df.VIP.isnull()) & (df.CryoSleep == True)] = df.loc[(df.VIP.isnull()) & (df.CryoSleep == True)].fillna({'VIP':False})

In [ ]:
vip = df.loc[(df.VIP ==True) & (df.CryoSleep == False)]
novip = df.loc[(df.VIP ==False) & (df.CryoSleep == False)]
vip[SERVICES].sum(axis=1).mean(), novip[SERVICES].sum(axis=1).mean()

In [ ]:
def get_VIP(item):
    mean = 2149.8388002552647
    if item > mean:
        return True
    else:
        return False

In [ ]:
df.VIP.loc[(df.VIP.isnull()) & (df.CryoSleep == False)] = df.loc[(df.VIP.isnull()) & (df.CryoSleep == False)][SERVICES].sum(axis=1).apply(get_VIP)

<p id="Cabin" style="background-color:#1A2D45;font-family:newtimeroman; color:white;text-align:center;border-radius:10px;font-size:24px">Missings in the Cabin features</p>

The similar technique for finding missing values for HomePlanet anf Destionation is used for Cabin features too!

In [ ]:
def deck_by_group(item):
    group_df = df.loc[df.Group == item]
    if len(group_df.loc[group_df.Deck.notnull()]) > 1:
        return group_df.Deck.describe().top
    else:
        return df.Deck.describe().top
        

def cabinnum_by_group(item):
    group_df = df.loc[df.Group == item]
    if len(group_df.loc[group_df.Cabin_num.notnull()]) > 1:
        return group_df.Cabin_num.mean().astype(int)
    else:
        return df.Cabin_num.mean().astype(int)
        

def side_by_group(item):
    group_df = df.loc[df.Group == item]
    if len(group_df.loc[group_df.Side.notnull()]) > 1:
        return group_df.Side.describe().top
    else:
        return df.Side.describe().top

In [ ]:
df['Deck'].loc[(df.Deck.isnull())] = df.loc[(df.Deck.isnull())]['Group'].apply(deck_by_group)
df['Cabin_num'].loc[(df.Cabin_num.isnull())] = df.loc[(df.Cabin_num.isnull())]['Group'].apply(cabinnum_by_group)

df['Side'].loc[(df.Side.isnull())]= df.loc[(df.Side.isnull())]['Group'].apply(side_by_group)


<p id="Age" style="background-color:#1A2D45;font-family:newtimeroman; color:white;text-align:center;border-radius:10px;font-size:24px">Missings in the Age</p>
If passenger in Group has another passengers which have notnull in Age column, the passenger with missing values will have mean value with rounded(from 28.83 => 29.0).

Otherwise, function will choose  mean values oof Age in main dataset

In [ ]:
def age_fixing(item):
    group = df.loc[df.Group == item]
    if len(group.loc[group.Age.notnull()]) > 1:
        return group.Age.describe()['mean'].round()
    else:
        return df.Age.describe()['mean'].round()

In [ ]:
df.Age.loc[(df.Age.isnull())& (df.Group.notnull())] = df.loc[(df.Age.isnull())& (df.Group.notnull())]['Group'].apply(age_fixing)

<p id="Services" style="background-color:#1A2D45;font-family:newtimeroman; color:white;text-align:center;border-radius:10px;font-size:24px">Missings in the Services</p>

<p style="background-color:#546981 ;font-family:newtimeroman;color:white;font-size:24px;text-align:center;border-radius:10px;">Missings in RoomService</p>

In [ ]:
def roomservice_fixing(item):
    group = df.loc[df.Group == item]
    if len(group.loc[group.RoomService.notnull()]) > 1:
        return int(group.RoomService.mean())
    else:
        return 0.0  # more than 8800 passenger did not use current service

In [ ]:
df.RoomService.loc[(df.RoomService.isnull())] = df.loc[(df.RoomService.isnull())]['Group'].apply(roomservice_fixing)

<p style="background-color:#546981 ;font-family:newtimeroman;color:white;font-size:24px;text-align:center;border-radius:10px;">Missings in FoodCourt</p>

In [ ]:
def food_fixing(item):
    group = df.loc[df.Group == item]
    if len(group.loc[group.FoodCourt.notnull()]) > 1:
        return int(group.FoodCourt.mean())
    else:
        return 0.0 # more than 8200 passengers did not use that 

In [ ]:
df.FoodCourt.loc[(df.FoodCourt.isnull())] = df.loc[(df.FoodCourt.isnull())]['Group'].apply(food_fixing)

<p style="background-color:#546981 ;font-family:newtimeroman;color:white;font-size:24px;text-align:center;border-radius:10px;">Missings in ShoppingMall</p>

In [ ]:
def shopping_fixing(item):
    group = df.loc[df.Group == item]
    if len(group.loc[group.ShoppingMall.notnull()]) > 1:
        return int(group.ShoppingMall.mean())
    else:
        return 0.0 # more than 8600 passengers did not use that 

In [ ]:
df.ShoppingMall.loc[(df.ShoppingMall.isnull())] = df.loc[(df.ShoppingMall.isnull())]['Group'].apply(shopping_fixing)

<p style="background-color:#546981 ;font-family:newtimeroman;color:white;font-size:24px;text-align:center;border-radius:10px;">Missings in Spa</p>

In [ ]:
def spa_fixing(item):
    group = df.loc[df.Group == item]
    if len(group.loc[group.Spa.notnull()]) > 1:
        return int(group.Spa.mean())
    else:
        return 0.0 # more than 8000 passengers did not use that 

In [ ]:
df.Spa.loc[(df.Spa.isnull())] = df.loc[(df.Spa.isnull())]['Group'].apply(spa_fixing)

<p style="background-color:#546981 ;font-family:newtimeroman;color:white;font-size:24px;text-align:center;border-radius:10px;">Missings in VRDeck</p>

In [ ]:
def vrdeck_fixing(item):
    group = df.loc[df.Group == item]
    if len(group.loc[group.VRDeck.notnull()]) > 1:
        return int(group.VRDeck.mean())
    else:
        return 0.0 # more than 8300 passengers did not use that 

In [ ]:
df.VRDeck.loc[(df.VRDeck.isnull())] = df.loc[(df.VRDeck.isnull())]['Group'].apply(vrdeck_fixing)

In [ ]:
missingno.matrix(df,figsize=(16,8), fontsize=12, color=(0.10, 0.18, 0.27)) #color in misingno is in RGB Percent
plt.show()

In [ ]:
df.isnull().sum()

<p id="outputs" style="background-color:#1A2D45;font-family:newtimeroman;color:white;font-size:24px;text-align:center;border-radius:10px;">Outputs</p>

In [ ]:
train = df.loc[df.Transported.notnull()]
train.to_csv("train_with_no_missings.csv", index=False)
test = df.loc[df.Transported.isnull()]
test.to_csv("test_with_no_missings.csv", index=False)

<a href="#top" class="btn btn-success btn-lg active" role="button" aria-pressed="true" style="color:#1A2D45;" data-toggle="popover" title="go to Colors">Go to TOP</a>